In [1]:
def op_checkmultisig(stack,z):
    if len(stack)<1:
        return False
    n = decode_num(stack.pop())
    if len(stack) <n+1:
        return False
    sec_pubkeys=[]
    for _ in range(n):
        sec_pubkeys.append(stack.pop())
    m = decode_num(stack.pop())
    if len(stack) <m+1:
        return False
    der_signatures = []
    for _ in range(m):
        der_signatures.append(stack.pop()[:-1])
    stack.pop()
    try:
        raise NotImplementedError
    except (ValueError, SyntaxError):
        return False
    return True



## 연습문제 8.1 op.py의 op_checkmultisig 함수를 작성하시오.

In [2]:
def op_checkmultisig(stack, z):
    if len(stack) < 1:
        return False
    n = decode_num(stack.pop())
    if len(stack) < n + 1:
        return False
    sec_pubkeys = []
    for _ in range(n):
        sec_pubkeys.append(stack.pop())
    m = decode_num(stack.pop())
    if len(stack) < m + 1:
        return False
    der_signatures = []
    for _ in range(m):
        der_signatures.append(stack.pop()[:-1])  # <1>
    stack.pop()  # <2>
    try:
        points = [S256Point.parse(sec) for sec in sec_pubkeys]
        sigs = [Signature.parse(der) for der in der_signatures]
        for sig in sigs:
            if len(points) ==0:
                return False
            while points:
                point = points.pop(0)
                if point.verify(z,sig):
                    break
        stack.append(encode_num(1))
          # <3>
    except (ValueError, SyntaxError):
        return False
    return True

In [3]:
from helper import encode_base58_checksum

In [4]:
h160 = bytes.fromhex('74d691da1574e6b3c192ecfb52cc8984ee7b6c56')
print(encode_base58_checksum(b'\x05'+h160))

3CLoMMyuoDQTPRD3XYZtCvgvkadrAdvdXh


## 연습문제 8.2. 20바이트의 hash160 값을 p2pkh 주소로 변환하는 h160_to_p2pkh_address 함수를 작성하시오

In [5]:
def h160_to_p2pkh_address(h160, testnet=False):
    if testnet:
        prefix = b'\x6f'
    else:
        prefix = b'\x00'
    return encode_base58_checksum(prefix+h160)

In [10]:
import hashlib

# Base58Check 인코딩 함수 (checksum 포함)
def encode_base58_checksum(b):
    checksum = hashlib.sha256(hashlib.sha256(b).digest()).digest()[:4]
    return encode_base58(b + checksum)

# Base58 인코딩
def encode_base58(b):
    alphabet = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    num = int.from_bytes(b, 'big')
    result = ''
    while num > 0:
        num, mod = divmod(num, 58)
        result = alphabet[mod] + result
    # 원래 데이터의 앞부분 0x00은 Base58에서는 '1'로 변환
    for byte in b:
        if byte == 0:
            result = '1' + result
        else:
            break
    return result

# 테스트 함수 정의
def test_h160_to_p2pkh_address1():
    # 임의의 20바이트 hash160 값
    h160 = bytes.fromhex('1f8ac10f23c5b5bc1167bda84b833e5c057a77d2')  # 예제 hash160
    testnet_h160 = bytes.fromhex('f0b34a4c8d26ee77d7b94df8f2b1f9dcf2ef7f3e')  # Testnet 예제

    # 테스트넷 주소 생성
    testnet_address = h160_to_p2pkh_address(testnet_h160, testnet=True)
    print(f"Testnet Address: {testnet_address}")

    # 메인넷 주소 생성
    mainnet_address = h160_to_p2pkh_address(h160, testnet=False)
    print(f"Mainnet Address: {mainnet_address}")

    # 예상 값 확인 (이 값을 다른 도구에서 계산하여 비교)
    expected_mainnet_address = "n3TfFPyYCFruMT7dWdeZcytae55BPrQEVW"
    expected_testnet_address = "13snAkgpFKeNREdrzeE5V1B2zVxAwQSiZi"

    assert mainnet_address == expected_mainnet_address, "Mainnet address mismatch"
    assert testnet_address == expected_testnet_address, "Testnet address mismatch"
    print("All tests passed!")

# 테스트 함수 실행
test_h160_to_p2pkh_address1()


NameError: name 'h160_to_p2pkh_address1' is not defined

## 연습문제 8.3. 20바이트의 hash160 값을 p2sh 주소로 변환하는 h160_to_p2sh_address 함수를 작성하시오

In [11]:
def h160_to_p2sh_address(h160,testnet=False):
    if testnet:
        prefix = b'\xc4'
    else:
        prefix = b'\x05'
    return encode_base58_checksum(prefix+h160)

In [12]:
from helper import hash256

In [13]:
modified_tx = bytes.fromhex('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a38\
5aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036\
b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98be\
c453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa0\
5de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b9\
5abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2\
788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c5687000000000\
1000000')

In [14]:
s256 = hash256(modified_tx)

In [15]:
z = int.from_bytes(s256, 'big')

In [16]:
print(hex(z))

0xe71bfa115715d6fd33796948126f40a8cdd39f187e4afb03896795189fe1423c


In [17]:
from ecc import S256Point, Signature
from helper import hash256

In [18]:
modified_tx = bytes.fromhex('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a38\
5aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036\
b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98be\
c453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa0\
5de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b9\
5abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2\
788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c5687000000000\
1000000')

In [19]:
h256 = hash256(modified_tx)

In [21]:
z = int.from_bytes(h256, 'big')
z

104533698797560496821845132535319230899254894228014507304915351202849162871356

In [22]:
sec = bytes.fromhex('022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff\
1295d21cfdb70')

In [23]:
der = bytes.fromhex('3045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a\
8fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4\
ee942a89937')

In [24]:
point = S256Point.parse(sec)
sig = Signature.parse(der)
print(point.verify(z, sig))

True


## 연습문제 8.4. 앞에 트랜잭션에서 두 번째 서명을 검증하시오.

In [30]:
>>> from io import BytesIO
>>> from ecc import S256Point, Signature
>>> from helper import hash256, int_to_little_endian
>>> from script import Script
>>> from tx import Tx, SIGHASH_ALL
>>> hex_tx = '0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6\
bd304d87221a000000db00483045022100dc92655fe37036f47756db8102e0d7d5e28b3beb83a8\
fef4f5dc0559bddfb94e02205a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4e\
e942a8993701483045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8eef\
53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e754022\
01475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103\
b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04\
d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f40090000\
0000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976\
a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da\
1574e6b3c192ecfb52cc8984ee7b6c568700000000'
>>> hex_sec = '03b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4b\
b71'
>>> hex_der = '3045022100da6bee3c93766232079a01639d07fa869598749729ae323eab8ee\
f53577d611b02207bef15429dcadce2121ea07f233115c6f09034c0be68db99980b9a6c5e75402\
2'
>>> hex_redeem_script = '475221022626e955ea6ea6d98850c994f9107b036b1334f18ca88\
30bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7f\
bdbd4bb7152ae'
>>> sec = bytes.fromhex(hex_sec)
>>> der = bytes.fromhex(hex_der)
>>> redeem_script = Script.parse(BytesIO(bytes.fromhex(hex_redeem_script)))
>>> stream = BytesIO(bytes.fromhex(hex_tx))
>>> tx_obj = Tx.parse(stream)
>>> s = int_to_little_endian(tx_obj.version, 4)

In [36]:

# >>> s += encode_varint(len(tx_obj.tx_ins))
# >>> i = tx_obj.tx_ins[0]
# >>> s += TxIn(i.prev_tx, i.prev_index, redeem_script, i.sequence).serialize()
# >>> s += encode_varint(len(tx_obj.tx_outs))
# >>> for tx_out in tx_obj.tx_outs:
# ...    s += tx_out.serialize()
# >>> s += int_to_little_endian(tx_obj.locktime, 4)
# >>> s += int_to_little_endian(SIGHASH_ALL, 4)
# >>> z = int.from_bytes(hash256(s), 'big')
# >>> point = S256Point.parse(sec)
# >>> sig = Signature.parse(der)
# >>> print(point.verify(z, sig))

## 연습문제 8.5. p2sh 트랜잭션을 검증할 수 있도록 sig_hash와 verify_input을 수정하시오.